In [8]:
%%writefile ds_utils.py
import pandas as pd
from pathlib import Path
from typing import List, Optional, Tuple

# ---------------- PATH HELPERS ----------------

def get_project_root() -> Path:
    """
    Returns the folder where this file/notebooks live.
    You can adjust this later if you move things into subfolders.
    """
    return Path(".").resolve()


def get_data_path(filename: str) -> Path:
    """
    Build a full path to a file in the data folder.
    For now we assume there is a 'data' subfolder next to the notebooks.
    """
    return get_project_root() / "data" / filename


# ---------------- DATA LOADING ----------------

def load_smmt_data(filename: str = "SMMT_raw.csv") -> pd.DataFrame:
    """
    Load SMMT car sales data from the data folder.
    """
    path = get_data_path(filename)
    df = pd.read_csv(path)
    return df


def load_dvla_data(filename: str = "df_VEH0160_UK.csv") -> pd.DataFrame:
    """
    Load DVLA / DfT VEH0160 registrations data from the data folder.
    Adjust encoding/sep if needed.
    """
    path = get_data_path(filename)
    # If you get Unicode errors, change encoding/sep as you already did.
    df = pd.read_csv(path, encoding="latin1")
    return df


# ---------------- BASIC CLEANING ----------------

def standardise_date_column(
    df: pd.DataFrame,
    date_col: str
) -> pd.DataFrame:
    """
    Parse a date column into pandas datetime.
    """
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    return df


def drop_cols(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    """
    Drop columns if they exist.
    """
    df = df.copy()
    existing = [c for c in cols if c in df.columns]
    return df.drop(columns=existing)


# ---------------- LAG FEATURES ----------------

def create_lag_features(
    df: pd.DataFrame,
    group_cols: List[str],
    date_col: str,
    target_col: str,
    lags: List[int]
) -> pd.DataFrame:
    """
    Create lagged features (e.g. 3/6/12-month lags) for a target variable
    within groups (e.g. by model or segment).
    """
    df = df.copy()
    df = df.sort_values(group_cols + [date_col])

    for lag in lags:
        lag_name = f"{target_col}_lag_{lag}"
        df[lag_name] = (
            df
            .groupby(group_cols, observed=True)[target_col]
            .shift(lag)
        )

    return df


# ---------------- TIME SPLIT ----------------

def time_based_split(
    df: pd.DataFrame,
    date_col: str,
    split_date: str
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split into train and test based on a calendar date.
    Test set includes rows on/after split_date.
    """
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

    split_ts = pd.to_datetime(split_date)
    train = df[df[date_col] < split_ts]
    test = df[df[date_col] >= split_ts]

    return train, test


# ---------------- FEATURES + MODEL ----------------

def make_X_y(
    df: pd.DataFrame,
    target_col: str,
    drop_cols_for_X: Optional[List[str]] = None
):
    """
    Split into X (features) and y (target), and build a preprocessing pipeline.
    """
    from sklearn.compose import ColumnTransformer
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.pipeline import Pipeline
    from sklearn.impute import SimpleImputer

    drop_cols_for_X = drop_cols_for_X or []

    feature_cols = [c for c in df.columns if c not in drop_cols_for_X + [target_col]]
    X = df[feature_cols].copy()
    y = df[target_col].copy()

    numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
    cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

    numeric_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="median")),
        ]
    )

    categorical_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore")),
        ]
    )

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_cols),
            ("cat", categorical_transformer, cat_cols),
        ]
    )

    return X, y, preprocessor


def build_rf_regressor(
    preprocessor,
    n_estimators: int = 300,
    max_depth: Optional[int] = None,
    random_state: int = 42
):
    """
    Build a RandomForestRegressor pipeline with preprocessing.
    """
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.pipeline import Pipeline

    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        n_jobs=-1,
        random_state=random_state,
    )

    pipe = Pipeline(
        steps=[
            ("preprocess", preprocessor),
            ("model", model),
        ]
    )

    return pipe


def evaluate_regression_model(model, X_train, y_train, X_test, y_test):
    """
    Fit a regression model and print simple metrics.
    """
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    import numpy as np

    model.fit(X_train, y_train)

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    mae_test = mean_absolute_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)

    print("Train RMSE:", round(rmse_train, 3))
    print("Test  RMSE:", round(rse_test, 3))
    print("Test  MAE :", round(mae_test, 3))
    print("Test  R2  :", round(r2_test, 3))

    return {
        "rmse_train": rmse_train,
        "rmse_test": rmse_test,
        "mae_test": mae_test,
        "r2_test": r2_test,
    }


Overwriting ds_utils.py


In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from ds_utils import (
    load_smmt_data,
    standardise_date_column,
    make_X_y,
    build_rf_regressor,
    evaluate_regression_model,
)

pd.set_option("display.max_columns", 100)
sns.set(style="whitegrid")

print("ds_utils imported OK")


ds_utils imported OK


In [17]:
import pandas as pd

dvla_df = pd.read_csv("/content/df_VEH0160_UK.csv", encoding="latin1")
print(dvla_df.shape)
dvla_df.head()


(61226, 50)


,BodyType,Make,GenModel,Model,Fuel,2025 Q3,2025 Q2,2025 Q1,2024 Q4,2024 Q3,2024 Q2,2024 Q1,2023 Q4,2023 Q3,2023 Q2,2023 Q1,2022 Q4,2022 Q3,2022 Q2,2022 Q1,2021 Q4,2021 Q3,2021 Q2,2021 Q1,2020 Q4,2020 Q3,2020 Q2,2020 Q1,2019 Q4,2019 Q3,2019 Q2,2019 Q1,2018 Q4,2018 Q3,2018 Q2,2018 Q1,2017 Q4,2017 Q3,2017 Q2,2017 Q1,2016 Q4,2016 Q3,2016 Q2,2016 Q1,2015 Q4,2015 Q3,2015 Q2,2015 Q1,2014 Q4,2014 Q3
0,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,PETROL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,10,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR AUTO,PETROL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,12,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Cars,ABARTH,ABARTH 500,500,PETROL,2,0,2,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,2,0,3,0,0,0,0,0,0,2,36,129,57,133,210
3,Cars,ABARTH,ABARTH 500,500 C,PETROL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,Cars,ABARTH,ABARTH 500,500 CUSTOM,PETROL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,37,128,210,16,0


In [20]:
# Columns that describe the car/model (non-time)
id_cols = ["BodyType", "Make", "GenModel", "Model", "Fuel"]

# All other columns are quarter columns like '2025 Q3', '2024 Q4', ...
time_cols = [c for c in dvla_df.columns if c not in id_cols]

print("Number of time columns:", len(time_cols))
print("First few time columns:", time_cols[:5])
print("Last few time columns:", time_cols[-5:])


Number of time columns: 45
First few time columns: ['2025 Q3', '2025 Q2', '2025 Q1', '2024 Q4', '2024 Q3']
Last few time columns: ['2015 Q3', '2015 Q2', '2015 Q1', '2014 Q4', '2014 Q3']


In [21]:
# Melt wide table (one row per model, many quarter columns)
# into long table (one row per model-quarter)
dvla_long = dvla_df.melt(
    id_vars=id_cols,
    value_vars=time_cols,
    var_name="YearQuarter",
    value_name="Registrations",
)

print("Long shape:", dvla_long.shape)
dvla_long.head()


Long shape: (2755170, 7)


,BodyType,Make,GenModel,Model,Fuel,YearQuarter,Registrations
0,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,PETROL,2025 Q3,0
1,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR AUTO,PETROL,2025 Q3,0
2,Cars,ABARTH,ABARTH 500,500,PETROL,2025 Q3,2
3,Cars,ABARTH,ABARTH 500,500 C,PETROL,2025 Q3,0
4,Cars,ABARTH,ABARTH 500,500 CUSTOM,PETROL,2025 Q3,0


In [22]:
# Drop rows where registrations are missing or zero if you want a cleaner panel
dvla_long = dvla_long.dropna(subset=["Registrations"])

# If you want to drop strict zeros:
# dvla_long = dvla_long[dvla_long["Registrations"] > 0]

print("After cleaning:", dvla_long.shape)


After cleaning: (2755170, 7)


In [25]:
from pandas import Timestamp
from pandas.tseries.offsets import MonthEnd

def parse_year_quarter(s):
    """
    Convert strings like '2025 Q3' to a Timestamp at quarter end
    (e.g. 2025-09-30 for 2025 Q3).
    Returns NaT if the format is unexpected.
    """
    if pd.isna(s):
        return pd.NaT

    s = str(s).strip()
    parts = s.split()

    # Expect exactly ['YYYY', 'Qn']
    if len(parts) != 2:
        return pd.NaT

    year_str, q_str = parts

    try:
        year = int(year_str)
    except ValueError:
        return pd.NaT

    q_str = q_str.upper()
    q_to_month = {"Q1": 3, "Q2": 6, "Q3": 9, "Q4": 12}
    month = q_to_month.get(q_str)
    if month is None:
        return pd.NaT

    # First day of that month, then move to end of month
    return Timestamp(year=year, month=month, day=1) + MonthEnd(0)

# Apply parser to YearQuarter
dvla_long["Date"] = dvla_long["YearQuarter"].apply(parse_year_quarter)

print(dvla_long[["YearQuarter", "Date"]].head(10))
print("Any missing dates? ->", dvla_long["Date"].isna().sum())


  YearQuarter       Date
0     2025 Q3 2025-09-30
1     2025 Q3 2025-09-30
2     2025 Q3 2025-09-30
3     2025 Q3 2025-09-30
4     2025 Q3 2025-09-30
5     2025 Q3 2025-09-30
6     2025 Q3 2025-09-30
7     2025 Q3 2025-09-30
8     2025 Q3 2025-09-30
9     2025 Q3 2025-09-30
Any missing dates? -> 0


In [26]:
dvla_long = (
    dvla_long
    .sort_values(["Make", "GenModel", "Model", "Date"])
    .reset_index(drop=True)
)

dvla_long.head()


,BodyType,Make,GenModel,Model,Fuel,YearQuarter,Registrations,YearQuarter_clean,Date
0,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,PETROL,2014 Q3,0,2014QQ3,2014-09-30
1,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,PETROL,2014 Q4,0,2014QQ4,2014-12-31
2,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,PETROL,2015 Q1,0,2015QQ1,2015-03-31
3,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,PETROL,2015 Q2,0,2015QQ2,2015-06-30
4,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,PETROL,2015 Q3,0,2015QQ3,2015-09-30


In [27]:
import pandas as pd

from ds_utils import (
    load_dvla_data,
    create_lag_features,
    get_data_path,
)

pd.set_option("display.max_columns", 100)

In [28]:
group_cols = ["Make", "GenModel", "Model"]
lags = [1, 2, 4]  # 1, 2, 4 quarters

dvla_with_lags = create_lag_features(
    dvla_long,
    group_cols=group_cols,
    date_col="Date",
    target_col="Registrations",
    lags=lags,
)

dvla_with_lags.head()


,BodyType,Make,GenModel,Model,Fuel,YearQuarter,Registrations,YearQuarter_clean,Date,Registrations_lag_1,Registrations_lag_2,Registrations_lag_4
0,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,PETROL,2014 Q3,0,2014QQ3,2014-09-30,NaN,NaN,NaN
1,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,PETROL,2014 Q4,0,2014QQ4,2014-12-31,0.0,NaN,NaN
2,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,PETROL,2015 Q1,0,2015QQ1,2015-03-31,0.0,0.0,NaN
3,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,PETROL,2015 Q2,0,2015QQ2,2015-06-30,0.0,0.0,NaN
4,Cars,ABARTH,ABARTH 124,124 GT MULTIAIR,PETROL,2015 Q3,0,2015QQ3,2015-09-30,0.0,0.0,0.0


In [29]:
import os

os.makedirs("/content/data", exist_ok=True)
print(os.listdir("/content"))

['.config', '__pycache__', 'ds_utils.py', 'df_VEH0160_UK.csv', 'data', 'sample_data']


In [30]:
out_path = get_data_path("dvla_with_lags_long.csv")
dvla_with_lags.to_csv(out_path, index=False)
print("Saved DVLA with lags to:", out_path)

Saved DVLA with lags to: /content/data/dvla_with_lags_long.csv
